In [1]:
import os
import pandas as pd 
import numpy as np 
import cv2
import __main__

from keras import regularizers
from keras.preprocessing import image 
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D, Dense, Dropout, Flatten 
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split



In [2]:
def get_image_value(path, dim): 
    '''This function will read an image and convert/resize'''
    img = image.load_img(path, target_size = dim)
    img = image.img_to_array(img)
    return img/255

def get_img_array(img_paths, dim): 
    '''This fucntion takes a list of image paths and returns np arrays.'''
    final_array = []
    from tqdm import tqdm
    for path in tqdm(img_paths):
        img = get_image_value(path, dim)
        final_array.append(img)
    final_array = np.array(final_array)  
    return final_array

def get_tts():
    '''This function will create a train test split'''  
   
    DIM =  (150,150) 
    np.random.seed(10)        
    
    faces_paths = [f'./Extracted_Data/00002/appleFace/{i}' for i in os.listdir(f'./Extracted_Data/00002/appleFace')]  
    faces_labels = [1 for i in range(len(faces_paths))]
    
    left_paths = [f'./Extracted_Data/00002/appleLeftEye/{i}' for i in os.listdir('./Extracted_Data/00002/appleLeftEye')] 
    left_labels = [2 for i in range(len(left_paths))]    
    
    right_paths = [f'./Extracted_Data/00002/appleRightEye/{i}' for i in os.listdir('./Extracted_Data/00002/appleRightEye')]
    
    np.random.shuffle(right_paths)
    right_paths = right_paths[:len(faces_paths)- 500]
    right_labels = [0 for i in range(len(right_paths))]

    np.random.shuffle(faces_paths)
    faces_paths = faces_paths[:len(left_paths)+150]
    right_paths = right_paths[:len(left_paths)+150]

    faces_labels = [1 for i in range(len(faces_paths))]
    left_labels = [2 for i in range(len(left_paths))]
    right_labels = [0 for i in range(len(right_paths))]
    paths = faces_paths + left_paths + right_paths
    labels = faces_labels + left_labels + right_labels
    x_train, x_test, y_train, y_test = train_test_split(paths, labels, stratify = labels, train_size = .90, random_state = 10)

    new_x_train = get_img_array(x_train, DIM)
    new_x_test = get_img_array(x_test, DIM)
    
    print('Train Value Counts')
    print(pd.Series(y_train).value_counts())
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('Test Value Counts')
    print(pd.Series(y_test).value_counts())
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('X Train Shape')
    print(new_x_train.shape)
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('X Test Shape')
    print(new_x_test.shape)
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

    y_train = np.array(y_train)
    y_test = np.array(y_test)
    y_test = to_categorical(y_test)
    y_train = to_categorical(y_train)
    tts = (new_x_train, new_x_test, y_train, y_test)
    
    return tts

x_train, x_test, y_train, y_test = get_tts()

100%|██████████| 12/12 [00:00<00:00, 571.70it/s]

Train Value Counts
1    50
2    50
dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Test Value Counts
1    6
2    6
dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
X Train Shape
(100, 150, 150, 3)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
X Test Shape
(12, 150, 150, 3)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [3]:
import numpy as np 
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D, Dense, Dropout, Flatten 
from tensorflow.keras.optimizers import Adam
from keras import regularizers
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd 
import matplotlib.pyplot as plt
import os
import pickle
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import cv2

def get_conv_model(dim = (150,150, 3)):
    '''This function will create and compile a CNN given the input dimension'''
    inp_shape = dim
    act = 'relu'
    drop = .25
    kernal_reg = regularizers.l1(.001)
    optimizer = Adam(lr = .0001)    
    model = Sequential() 
    model.add(Conv2D(64, kernel_size=(3,3),activation=act, input_shape = inp_shape, 
                     kernel_regularizer = kernal_reg,
                     kernel_initializer = 'he_uniform',  padding = 'same', name = 'Input_Layer'))
    model.add(MaxPooling2D(pool_size=(2, 2),  strides = (3,3)))
    model.add(Conv2D(64, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (3,3))) 
    model.add(Conv2D(128, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(Conv2D(128, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (3,3)))  
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(drop))
    model.add(Dense(3, activation='softmax', name = 'Output_Layer'))
    model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return model 

In [4]:

#prevents overfitting and saves models every time the validation loss improves
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=10, min_delta = .00075)
model_checkpoint = ModelCheckpoint('ModelWeights.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 2, mode = 'min')
epochs = 10
batch_size = 2
model = get_conv_model()
model_history = model.fit(x_train, y_train, batch_size = batch_size,
            epochs = epochs, 
     callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)

Epoch 1/10


C:\Users\Justin_PC_W10\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


50/50 [==============================] - ETA: 0s - loss: 11.8139 - accuracy: 0.6300
Epoch 1: val_loss improved from inf to 11.18729, saving model to ModelWeights.h5
50/50 [==============================] - 4s 64ms/step - loss: 11.8139 - accuracy: 0.6300 - val_loss: 11.1873 - val_accuracy: 0.6667 - lr: 1.0000e-04
Epoch 2/10
50/50 [==============================] - ETA: 0s - loss: 10.8089 - accuracy: 0.8500
Epoch 2: val_loss improved from 11.18729 to 10.23607, saving model to ModelWeights.h5
50/50 [==============================] - 3s 61ms/step - loss: 10.8089 - accuracy: 0.8500 - val_loss: 10.2361 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 3/10
49/50 [============================>.] - ETA: 0s - loss: 9.8808 - accuracy: 0.9694
Epoch 3: val_loss improved from 10.23607 to 9.43534, saving model to ModelWeights.h5
50/50 [==============================] - 3s 59ms/step - loss: 9.8719 - accuracy: 0.9700 - val_loss: 9.4353 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 4/10
50/50 [============

In [15]:
cascPath = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"

# Create the haar cascade
faceCascade = cv2.CascadeClassifier(cascPath)

def non_max_suppression(boxes, overlapThresh= .5):
    
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []
    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
    # initialize the list of picked indexes	
    pick = []
    # grab the coordinates of the bounding boxes
    x1, y1, x2, y2 = boxes[:,0], boxes[:,1], boxes[:,2], boxes[:,3]    
    # compute the area of the bounding boxes and sort the bounding
    # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1, yy1, xx2, yy2 = np.maximum(x1[i], x1[idxs[:last]]), np.maximum(y1[i], y1[idxs[:last]]), np.minimum(x2[i], x2[idxs[:last]]), np.minimum(y2[i], y2[idxs[:last]])
        # compute the width and height of the bounding box
        w, h = np.maximum(0, xx2 - xx1 + 1), np.maximum(0, yy2 - yy1 + 1)
        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]
        # delete all indexes from the index list that have
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))
    # return only the bounding boxes that were picked using the
    # integer data type
    return pick

def get_img_prediction_bounding_box(path, model, dim):
    '''This function will create a bounding box over what it believes is a 
    Face given the image path, dimensions, and model used to detect the Face.  
    Dimensions can be found within the Var.py file.  This function is still 
    being used as I need to apply non-max suppresion to create only one bounding box'''
    img = get_image_value(path, dim)   
    img = img.reshape(1, img.shape[0], img.shape[1], 3)
    pred = model.predict(img)[0]
    category_dict = {0: 'LeftEye', 1: 'Face', 2: 'RightEye'}
    cat_index = np.argmax(pred)
    cat = category_dict[cat_index]
    print(f'{path}\t\tPrediction: {cat}\t{int(pred.max()*100)}% Confident')

    #speed up cv2
    cv2.setUseOptimized(True)
    cv2.setNumThreads(10) #change depending on your computer
    img = cv2.imread(path)
    clone = img.copy()
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(img)
    ss.switchToSelectiveSearchFast()

    rects = ss.process() 
    windows = []
    locations = []
    
    print(f'Creating Bounding Boxes for {path}')
    for x, y, w,h in rects[:1001]: 
        startx, starty, endx, endy = x, y, x+w, y+h 
        roi = img[starty:endy, startx:endx]
        roi = cv2.resize(roi, dsize =dim, interpolation = cv2.INTER_CUBIC)
        windows.append(roi)
        locations.append((startx, starty, endx, endy))
    
    gray = cv2.cvtColor(clone, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the image
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
        )
    
    windows = np.array(windows)
    windows = windows.reshape(windows.shape[0], windows.shape[1], windows.shape[2], 3)
    windows = np.array(windows)
    locations = np.array(locations)
    predictions = model.predict(windows)
    nms = non_max_suppression(locations)
    bounding_cnt = 0
        
    if len(faces) > 0:
    
        # Draw a rectangle around the faces
        for (x1, y1, w, h) in faces:
            print("Found {0} faces!".format(len(faces)))
            startx, starty, endx, endy = (x1, y1, w, h)
            cv2.rectangle(clone, (x1, y1), (x1+w, y1+h), (0, 255, 0), 2)
            text = f'{category_dict[np.argmax(predictions[cat_index])]}: {int(predictions[cat_index].max()*100)}%'
            cv2.putText(clone, text, (startx, starty+15), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,255,0),2)
            cv2.imshow("Faces found", clone)
            #cv2.imshow(f'Test', np.hstack([clone, clone2]))
            #cv2.rectangle(clone, (startx, starty), (endx, endy), (0,0,255), 2)
        
        cv2.waitKey(1)
    
    else:
        
        cv2.imshow("No Faces found", clone)
           
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    cv2.waitKey(0)
    ss.clear()
    return predictions

#NORMAL MODEL
dim = (150, 150, 3)    
normal_model = get_conv_model(dim)
normal_model.load_weights('ModelWeights.h5') #path to the model weights
test_folder = './Extracted_Data/00000_Tests' #folder where you will put your images to test
predictions = []
for idx, i in enumerate([i for i in os.listdir(test_folder) if i != 'ipynb_checkpoints']):
    img_path = f'{test_folder}/{i}'
    pred = get_img_prediction_bounding_box(img_path, normal_model, dim = (150,150))
    predictions.append(pred)
    

./Extracted_Data/00000_Tests/aTWO.JPG		Prediction: Face	100% Confident
Creating Bounding Boxes for ./Extracted_Data/00000_Tests/aTWO.JPG
Found 3 faces!
Found 3 faces!
Found 3 faces!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
./Extracted_Data/00000_Tests/Face_00000.jpg		Prediction: Face	100% Confident
Creating Bounding Boxes for ./Extracted_Data/00000_Tests/Face_00000.jpg
Found 1 faces!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
./Extracted_Data/00000_Tests/X.jpg		Prediction: Face	99% Confident
Creating Bounding Boxes for ./Extracted_Data/00000_Tests/X.jpg
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
